# Fit $\beta_{ij}$ for cross-influence parameter of Hexane and Ethanol

This notebook has te purpose of showing how to optimize the $\beta_{ij}$ correction with SGTPy.

First it's needed to import the necessary modules

In [4]:
import numpy as np

from SGTPy import component, mixture, saftvrmie
from SGTPy.equilibrium import bubblePy
from SGTPy.fit import fit_beta

Now that the functions are available it is necessary to create the mixture, set up the interacion parameter $k_{ij}$ and the create the model.

In [2]:
ethanol = component('ethanol2C', ms = 1.7728, sigma = 3.5592 , eps = 224.50,
                    lambda_r = 11.319, lambda_a = 6., eAB = 3018.05, rcAB = 0.3547,
                    rdAB = 0.4, sites = [1,0,1], cii= 5.3141080872882285e-20)

hexane = component('hexane', ms = 1.96720036, sigma = 4.54762477, eps = 377.60127994, 
                   lambda_r = 18.41193194, cii = 3.581510586936205e-19)

mix = mixture(hexane, ethanol)

# kij fitted to experimental data
kij = 0.011818492037463553
Kij = np.array([[0, kij], [kij, 0]])
mix.kij_saft(Kij)
eos = saftvrmie(mix)

The ```eos``` object has all the necessary methods to compute phase equilibria and interfacial properties with SGT.

Now the phase equilibria is computed at the experimental conditions.

In [3]:
# Experimental IFT obtained from Jimenez et at, J. Chem. Eng. Data, vol. 45, no. 5, pp. 862–866, 2000.

# Experiment temperature
T = 298.15 # K

# Experimental liquid composition
x1 = np.array([0.0448, 0.1625, 0.2116, 0.2559, 0.3034, 0.3745, 0.4366, 0.47, 0.4974, 0.5863, 0.6452, 
                 0.6841, 0.7584, 0.7963, 0.8500, 0.9067, 0.9347]) 

# Experimental interfacial tension in mN/m
iftexp = np.array([21.10, 19.74, 19.26, 18.98, 18.83, 18.56, 18.33, 18.28, 18.16, 18.12, 18.06, 18.02, 17.97, 17.95,
                  17.93, 17.91, 17.89]) 

In [4]:
# First it is necessary to compute the phase equilibria at given experimental conditions

n = len(x1)

X = np.array([x1, 1-x1])
Y = np.zeros_like(X)
P = np.zeros_like(x1)
vl = np.zeros_like(x1)
vv = np.zeros_like(x1)

#computing phase equilibria
i = 0
sol = bubblePy(X[:, i], 10000., X[:, i], T, eos, full_output = True)
Y[:, i], P[i] = sol.Y, sol.P
vl[i], vv[i] = sol.v1, sol.v2

for i in range(1, n):
    sol = bubblePy(Y[:, i-1], P[i-1], X[:, i], T, eos, full_output = True, v0 = [vl[i-1], vv[i-1]])
    Y[:, i], P[i] = sol.Y, sol.P
    vl[i], vv[i] = sol.v1, sol.v2

# computing density vectors
rhox = X/vl
rhoy = Y/vv

To optimize $\beta_{ij}$ is necessary to create a tupple with the density vectors, equilibrium temperature and equilibrium pressure. This is done with ```EquilibriumInfo = (rhox, rhoy, T*np.ones_like(x1), P)```

Then the function ```fit_beta``` requires the bounds to start the minimization procedure, the experimental interfacial tension, the equilibrium information and the model.

In [ ]:
# Beta optimization can be slow
EquilibriumInfo = (rhoy, rhox, T*np.ones_like(x1), P)
beta_bounds = (0.01, 0.1)
fit_beta(beta_bounds, iftexp, EquilibriumInfo, eos)

For more information just run:
```fit_beta?```